# Ćwiczenie 1
## Zadanie 1
Izabela Stobiecka

Zacznijmy od zimportowania wszystkich pakietów użytych w klasie i w przykładzie:

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from statistics import mode
from collections import Counter

### Klasa moj_model 

In [2]:
class moj_model(object):
    """
    Klasa wykonująca algorytm klasyfikatora kNN z adaptacyjnym doborem parametru k. 
    Odległość jest wyznaczana za pomocą metryki euklidesowej.
    """
    
    def __init__(self, n_neighbors = 0, kmax = 10, shuffle = False, nsplits = 5):
        """
        :param n_neighbors: int, k najbliższych sąsiadów które chcemy brać pod uwagę przy predykcji,
                            jeśli = 0 klasyfikator sam dobiera parametr k 
        :param kmax:        int, maksymalne k najbliższych sąsiadów dla których sprawdzany jest wynik cross validation,
                            jeśli = 0, sprawdzone zostanie każde k
        :param shuffle:     boolean, gdy True- zbiór treningowy jest tasowany przed dokonaniem cross validation
        :param nsplits:     int, liczba podziałów przy cross validation
        """
        self.n_neighbors = n_neighbors
        self.nsplits = nsplits
        self.kmax = kmax
        self.shuffle = shuffle
        
    def __typeCheck(self, X):
        """
        Zamiana pandas data frame na numpy ndarray (jeśli konieczna)
        """     
        if type(X) == pd.core.frame.DataFrame:
            X = pd.DataFrame.to_numpy(X)
        return X

        
    def __prefit(self, X_train, y_train):
        """
        Dokonuje wstępnego przygotowania danych do predykcji
        :param X_train: numpy.ndarray, tablica z wartościami zmiennej objaśniajacej zbioru treningowego 
        :param y_train: numpy.ndarray, tablica z wartościami klas zbioru treningowego
        :param size:    int, liczba rekordów w X_train
        :param dim:     int, liczba atrybutów w X_train
        """
        self.X_train = self.__typeCheck(X_train)
        self.y_train = y_train         
        self.size = len(self.X_train) 
        self.dim = len(self.X_train[0]) 
    
    def fit(self, X_train, y_train):
        """
        Dopasowuje model do zbioru treningowego.
        Jeśli n_neighbors = 0, dokonuje doboru liczby najbliższych sąsiadów na podstawie wyniku accuracy
        w cross validacji na zbiorze treningowym (wybierane jest k dla którego najniższy wynik w CV jest największy)
        :param k : dopasowana do danych treningowych liczba najbliższych sąsiadów
        """   
        self.__prefit(X_train, y_train) 
        
        if self.kmax == 0:
            self.kmax = self.size      
              
        if self.n_neighbors == 0:
            kf = KFold(n_splits = self.nsplits, shuffle = self.shuffle) 
            k_array = np.full(self.kmax, 0.0)
            indexk = 0

            for k in range(1,self.kmax + 1):
                self.k = k
                scores = np.full(self.nsplits, 0.0)
                indexCV = 0
                kf.get_n_splits(self.X_train)

                for train_index, test_index in kf.split(self.X_train):
                    X_trainCV, X_testCV = self.X_train[train_index], self.X_train[test_index]
                    y_trainCV, y_testCV = self.y_train[train_index], self.y_train[test_index]
                    self.__prefit(X_trainCV, y_trainCV) 
                    pr = self.predict(X_testCV)
                    acS = accuracy_score(y_testCV, pr)
                    scores[indexCV] = acS
                    indexCV += 1
                    #Powrót do wejściowego zbioru treningowego:
                    self.__prefit(X_train, y_train)
                
                scores = np.sort(scores)
                k_array[indexk] = scores[0]
                indexk += 1

            self.k = np.argmax(k_array) + 1
        else:
            self.k = self.n_neighbors
        
    def predict(self, X_test):
        """
        Dokonuje predykcji na zbiorze testowym.
        Dla każdego rekordu w X_test oblicza odległości w metryce euklidesowej do wszystkich elementów
        zbioru treningowego i wybiera k najmniejszych z nich. Następnie oblicza modę z klas k najbliższych
        sąsiadów i zwraca ją jako predykowaną klasę. 
        :param size_test:     int, rozmiar zbioru testowego
        """
        self.X_test = self.__typeCheck(X_test)
        self.size_test = len(self.X_test)
        self.y_test = np.full(self.size_test, 0)
        distances = np.full((self.size,2),0.0)
        indexq = 0
        for q in self.X_test:
            index = 0
            for a in self.X_train:                          
                dist = np.linalg.norm(a-q)
                distances[index,0] = dist
                distances[index,1] = self.y_train[index]
                index += 1 
            distances = distances[distances[:,0].argsort()]
            SmallestDistances = distances[0:self.k]
            #cl = mode(SmallestDistances[:,1])- ta funkcja niestety nie radzi sobie z więcej niż jedną modą
            cl = Counter(SmallestDistances[:,1]).most_common(1)[0][0]
            self.y_test[indexq] = cl
            indexq += 1
        return self.y_test
        

### Przykład działania
Ładujemy zbiór danych o irysach.

In [3]:
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target
df = pd.DataFrame(X) 

Dokonujemy rozdzielenia danych na zbiór treningowy i testowy.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

Inicjalizujemy nasz model (ustawiłam `shuffle = True`, ponieważ zbiór danych irysy jest pogrupowany ze względu na klasy,
co przeszkadza w skutecznej cross validacji). Dopasowujemy model do danych i dokonujemy predykcji na zbiorze testowym.

In [5]:
kNN = moj_model(shuffle = True)
kNN.fit(X_train, y_train)
pr = kNN.predict(X_test)

Sprawdzamy wynik accuracy i to, jaką liczbę k najbliższych sąsiadów dopasował nasz model. Następnie wywołujemy macierz pomyłek.

In [6]:
print('Otrzymaliśmy accuracy: ',accuracy_score(y_test, pr), 'przy k najbliższych sąsiadów równym', kNN.k)

Otrzymaliśmy accuracy:  0.9666666666666667 przy k najbliższych sąsiadów równym 4


In [7]:
pd.DataFrame(confusion_matrix(y_test, pr))

,0,1,2
0,13,0,0
1,0,5,1
2,0,0,11


### Porównanie z klasyfikatorem z pakietu sklearn
Porównajmy teraz nasz model z klasyfikatorem z pakietu sklearn przy tym samym k:

In [8]:
from sklearn.neighbors import KNeighborsClassifier

kNN2 = KNeighborsClassifier(n_neighbors = kNN.k)
kNN2.fit(X_train, y_train)
pr2 = kNN2.predict(X_test)

In [9]:
print('Otrzymaliśmy accuracy: ',accuracy_score(y_test, pr2), 'przy k najbliższych sąsiadów równym', kNN.k)

Otrzymaliśmy accuracy:  0.9666666666666667 przy k najbliższych sąsiadów równym 4


In [10]:
pd.DataFrame(confusion_matrix(y_test, pr2))

,0,1,2
0,13,0,0
1,0,5,1
2,0,0,11


Zazwyczaj oba klasyfikatory dokonują ten samej predykcji (czasem zdarza się różnica w pojedynczej predykcji, która wynika prawdopodobnie z inaczej obliczonej mody). Porównajmy jeszcze czas wykonania. 

In [11]:
kNN = moj_model(n_neighbors = kNN.k)
kNN.fit(X_train, y_train)
%timeit kNN.predict(X_test)

30.4 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
kNN2 = KNeighborsClassifier(n_neighbors = kNN.k)
kNN2.fit(X_train, y_train)
%timeit pr2 = kNN2.predict(X_test)

2.18 ms ± 301 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Jak można było się spodziewać, klasyfikator z pakietu sklearn dokonał predykcji znacząco szybciej.

In [ ]:
print('he;llo')